In [101]:
#Import packages
import sys, os

import numpy as np

from sklearn import preprocessing

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils

In [102]:
#Load data
file_dir='data'
data_csv='Data2.csv'
label_csv='Labels2.csv'

data_path = os.path.join(file_dir, data_csv)
label_path = os.path.join(file_dir, label_csv)

data_mat_unNorm = ( np.loadtxt(open(data_path, "rb"), delimiter=",") ).astype('float32')
label_mat = ( np.loadtxt(open(label_path, "rb"), delimiter=",") ).astype('float32')

print np.shape(data_mat_unNorm)
print np.shape(label_mat)

(333, 165)
(333,)


In [103]:
# Normalize values to 0-1

#Set up min max scaler
scaler = preprocessing.MinMaxScaler()
print scaler 
print

#Fit scaler to data
data_mat = scaler.fit_transform(data_mat_unNorm)

print data_mat.max()
print data_mat.min()


MinMaxScaler(copy=True, feature_range=(0, 1))

1.0
0.0


In [104]:
# ==========================================================
# Divide up sets, set random number seed
# ==========================================================

#Set training set ratio to the entire sample
TRAINING_RATIO = 0.8

#Get the sample and feature size
sample_size = np.shape(data_mat)[0]
feature_size = np.shape(data_mat)[1]

print "Number of samples: %d" % sample_size
print "Number of features: %d\n" % feature_size

#Get a random set of indeces for random sampling
a = np.arange(sample_size)
np.random.shuffle(a)

#Take the first set of random integers
cut_off_idx = int(sample_size*TRAINING_RATIO)
training_set_indeces = a[0:cut_off_idx] 
validating_set_indeces = a[cut_off_idx:]


#Set up empty matrices for the two matrices
X_train = np.zeros((cut_off_idx,feature_size),dtype='float32')
y_train = np.zeros((cut_off_idx,1),dtype='float32')
X_test = np.zeros((sample_size-cut_off_idx,feature_size),dtype='float32')
y_test = np.zeros((sample_size-cut_off_idx,1),dtype='float32')


#Iterate through training indeces to initialize
for i in range(0,len(training_set_indeces)):
    tr_idx = training_set_indeces[i]
    X_train[i,:] = data_mat[tr_idx,:]
    y_train[i] = label_mat[tr_idx]
#Iterate thorugh the cross validation set to initalize
for j in range(0,len(validating_set_indeces)):
    cv_idx = validating_set_indeces[j]
    X_test[j,:] = data_mat[cv_idx,:]
    y_test[j] = label_mat[cv_idx]

    
#Flatten the 
y_train = y_train.flatten()
y_test = y_test.flatten()
    
print "Training set ratio: %f\n" % TRAINING_RATIO
print "X_train shape:",
print np.shape(X_train)
print "X_test shape:",
print np.shape(X_test)
print "y_train shape:",
print np.shape(y_train)
print "y_test shape:",
print np.shape(y_test)


#Fix a random number generator seed for reproducibility
seed = 7
np.random.seed(seed)


Number of samples: 333
Number of features: 165

Training set ratio: 0.800000

X_train shape: (266, 165)
X_test shape: (67, 165)
y_train shape: (266,)
y_test shape: (67,)


In [105]:
# ==========================================================
# "One hot encode" the label vectors
# ==========================================================
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

print np.shape(y_train)
print np.shape(y_test)
print num_classes

(266, 3)
(67, 3)
3


In [106]:
# ==========================================================
# Define multi-layer perceptron model
# ==========================================================

def baseline_model():
    #Create model
    model = Sequential()
    
    #Add a single hidden layer with the same number of neurons as there are inputs
    model.add(Dense(feature_size/2, input_dim=feature_size, kernel_initializer='normal', activation='relu'))
    
    #Add dropout layer
    model.add(Dropout(0.1))
    
    #Add 5 more layers to simulate the 6 cortical layers
    for temp_i in range(0,5):
        model.add(Dense(feature_size/2, input_dim=feature_size/2, kernel_initializer='normal', activation='relu'))
    
    #Add output layer
    model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))
    #Compile the model
    #    logorithmic loss function - 'categorical_crossentropy'
    #    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
    

In [107]:
# ==========================================================
# Build and test model
# ==========================================================

#Build the model
model = baseline_model()

#Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=200, verbose=1)

#Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))


Train on 266 samples, validate on 67 samples
Epoch 1/100
266/266 [==============================] - 0s - loss: 1.0983 - acc: 0.4887 - val_loss: 1.0959 - val_acc: 0.4328
Epoch 2/100
266/266 [==============================] - 0s - loss: 1.0952 - acc: 0.4887 - val_loss: 1.0914 - val_acc: 0.4328
Epoch 3/100
266/266 [==============================] - 0s - loss: 1.0903 - acc: 0.4887 - val_loss: 1.0843 - val_acc: 0.4328
Epoch 4/100
266/266 [==============================] - 0s - loss: 1.0824 - acc: 0.4887 - val_loss: 1.0723 - val_acc: 0.4328
Epoch 5/100
266/266 [==============================] - 0s - loss: 1.0689 - acc: 0.4887 - val_loss: 1.0510 - val_acc: 0.4328
Epoch 6/100
266/266 [==============================] - 0s - loss: 1.0449 - acc: 0.4887 - val_loss: 1.0142 - val_acc: 0.4328
Epoch 7/100
266/266 [==============================] - 0s - loss: 1.0045 - acc: 0.4887 - val_loss: 0.9550 - val_acc: 0.4328
Epoch 8/100
266/266 [==============================] - 0s - loss: 0.9390 - acc: 0.4887 

266/266 [==============================] - 0s - loss: 0.1283 - acc: 0.9624 - val_loss: 0.3249 - val_acc: 0.9254
Epoch 67/100
266/266 [==============================] - 0s - loss: 0.1031 - acc: 0.9812 - val_loss: 0.3536 - val_acc: 0.8806
Epoch 68/100
266/266 [==============================] - 0s - loss: 0.1348 - acc: 0.9662 - val_loss: 0.3078 - val_acc: 0.9104
Epoch 69/100
266/266 [==============================] - 0s - loss: 0.1115 - acc: 0.9699 - val_loss: 0.2828 - val_acc: 0.9104
Epoch 70/100
266/266 [==============================] - 0s - loss: 0.1182 - acc: 0.9737 - val_loss: 0.2878 - val_acc: 0.9104
Epoch 71/100
266/266 [==============================] - 0s - loss: 0.1177 - acc: 0.9774 - val_loss: 0.2970 - val_acc: 0.9104
Epoch 72/100
266/266 [==============================] - 0s - loss: 0.1122 - acc: 0.9774 - val_loss: 0.2763 - val_acc: 0.9254
Epoch 73/100
266/266 [==============================] - 0s - loss: 0.1096 - acc: 0.9699 - val_loss: 0.2783 - val_acc: 0.9104
Epoch 74/100
